In [2]:
#Sistemas de recomendación

In [6]:
#Users
#Items
#1 User---->N Items
#1 Item ----> N User


In [8]:
import numpy as np
import pandas as pd
import scipy.stats as sts

In [9]:
df = pd.DataFrame({'item': ['manzana', 'sandia', 'pera', 'pizza', 'perrito', 'hamburguesa',
                            'ensalada', 'hummus', 'chuleton', 'longaniza', 'lubina'],
                   'a': [5, 5, 0, 0, 0, 0, 4, 0, 0, 0, 0],
                   'b': [4, 3, 0, 5, 4, 4, 0, 0, 4, 4, 0],
                   'c': [4, 3, 3, 0, 0, 0, 3, 4, 2, 0, 2],
                   'd': [1, 2, 1, 5, 5, 5, 0, 2, 4, 3, 0],
                   'e': [5, 5, 5, 0, 0, 0, 4, 5, 0, 0, 0],
                   'f': [5, 5, 0, 0, 0, 0, 4, 5, 0, 0, 0]})
df.set_index('item', inplace=True)
df

,a,b,c,d,e,f
item,,,,,,
manzana,5,4,4,1,5,5
sandia,5,3,3,2,5,5
pera,0,0,3,1,5,0
pizza,0,5,0,5,0,0
perrito,0,4,0,5,0,0
hamburguesa,0,4,0,5,0,0
ensalada,4,0,3,0,4,4
hummus,0,0,4,2,5,5
chuleton,0,4,2,4,0,0


In [10]:

df_bin = (df>0).astype(int)
df_bin

,a,b,c,d,e,f
item,,,,,,
manzana,1,1,1,1,1,1
sandia,1,1,1,1,1,1
pera,0,0,1,1,1,0
pizza,0,1,0,1,0,0
perrito,0,1,0,1,0,0
hamburguesa,0,1,0,1,0,0
ensalada,1,0,1,0,1,1
hummus,0,0,1,1,1,1
chuleton,0,1,1,1,0,0


In [13]:
#interesecciones
#jaccard loss son similares

In [24]:
jaccard = lambda a,b : len(a.intersection(b)) / (len(a.union(b))+0.00000000001)


In [25]:
jaccard(set([1,2,3]),set([7,8,9]))

0.0

In [26]:
set(df_bin[df_bin['a']>0].index)

{'ensalada', 'manzana', 'sandia'}

In [28]:
for u0 in df_bin.columns:
    for u1 in df_bin.columns:
        if u0 != u1:
            items0=set(df_bin[df_bin[u0]>0].index)
            items1=set(df_bin[df_bin[u1]>0].index)
            print("Sim({},{}) = {}".format(u0,u1,jaccard(items0,items1)))

Sim(a,b) = 0.24999999999968753
Sim(a,c) = 0.4285714285708163
Sim(a,d) = 0.1999999999998
Sim(a,e) = 0.5999999999988
Sim(a,f) = 0.749999999998125
Sim(b,a) = 0.24999999999968753
Sim(b,c) = 0.2727272727270248
Sim(b,d) = 0.7777777777769137
Sim(b,e) = 0.1999999999998
Sim(b,f) = 0.22222222222197532
Sim(c,a) = 0.4285714285708163
Sim(c,b) = 0.2727272727270248
Sim(c,d) = 0.45454545454504136
Sim(c,e) = 0.7142857142846939
Sim(c,f) = 0.571428571427755
Sim(d,a) = 0.1999999999998
Sim(d,b) = 0.7777777777769137
Sim(d,c) = 0.45454545454504136
Sim(d,e) = 0.3999999999996
Sim(d,f) = 0.2999999999997
Sim(e,a) = 0.5999999999988
Sim(e,b) = 0.1999999999998
Sim(e,c) = 0.7142857142846939
Sim(e,d) = 0.3999999999996
Sim(e,f) = 0.7999999999984
Sim(f,a) = 0.749999999998125
Sim(f,b) = 0.22222222222197532
Sim(f,c) = 0.571428571427755
Sim(f,d) = 0.2999999999997
Sim(f,e) = 0.7999999999984


In [29]:
#Necessitamos un top K de vecinos que mas se parecen

In [31]:

u0 = 'a'

neig = []
for u1 in df_bin.columns:
    if u0 != u1:
        items0=set(df_bin[df_bin[u0]>0].index)
        items1=set(df_bin[df_bin[u1]>0].index)
        neig.append((u1,jaccard(items0,items1)))
        print("Sim({},{}) = {}".format(u0,u1,jaccard(items0,items1)))

Sim(a,b) = 0.24999999999968753
Sim(a,c) = 0.4285714285708163
Sim(a,d) = 0.1999999999998
Sim(a,e) = 0.5999999999988
Sim(a,f) = 0.749999999998125


In [32]:
neig

[('b', 0.24999999999968753),
 ('c', 0.4285714285708163),
 ('d', 0.1999999999998),
 ('e', 0.5999999999988),
 ('f', 0.749999999998125)]

In [40]:
top = sorted(neig, key = lambda x: -x[1])[0:2]

In [41]:
top

[('f', 0.749999999998125), ('e', 0.5999999999988)]

In [54]:
# item -> score
recomendaciones = {}

for n, similarity in top:
    for item in df_bin[df_bin[n]>0].index:
        if item not in set(df_bin[df_bin['a']>0].index):
            if item in recomendaciones:                
                recomendaciones[item] += (similarity)
            else:
                recomendaciones[item] = similarity

In [55]:
recomendaciones

{'hummus': 1.3499999999969252, 'pera': 0.5999999999988}

In [56]:
#jaccard funciona con binarios
#pero no si tenemos la matriz puntuada

In [57]:
sts.pearsonr?

In [58]:
u0='a'
for u1 in df.columns:
    if u0!=u1:
        print(u0,u1, sts.pearsonr(df[u0],df[u1]))

a b (-0.014025268244765073, 0.9673537305875948)
a c (0.5630164352612619, 0.07133112524015639)
a d (-0.47773066916197693, 0.13724307333312027)
a e (0.6402913757205406, 0.033822514649123954)
a f (0.789823242704962, 0.0038279681529224444)
